In [1]:
import pandas as pd
import numpy as np
import time
from datetime import datetime
import warnings
import duckdb
import os
from bs4 import BeautifulSoup
import requests
import unicodedata
import smtplib
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart

pd.set_option('display.max_columns', None)
warnings.filterwarnings("ignore")

year = 2025
now = str(datetime.now().date())
print(f"Today's date:", now)

cwd = os.path.abspath(os.getcwd()).replace("\\", "/")
if cwd.startswith("C:/Users/Rodolfo/"):
    RUN_LOCATION = "local"
else:
    RUN_LOCATION = "cloud"
    import cloudscraper

print("Current working dir:", cwd)
print("RUN_LOCATION =", RUN_LOCATION)

Today's date: 2025-12-07
Current working dir: C:/Users/Rodolfo/Jupyter_files/FantasyBasketball/notebooks
RUN_LOCATION = local


In [2]:
def email(error, game_id):
    
    # Email details
    sender_email = "rodolfoe7157@gmail.com"
    receiver_email = "rodolfoe7157@gmail.com"
    password = "cqgu bfey cnyx sfue"  # See note below

    subject = "NBA gamelog_rtrvr error"
    body = f"game_id: {game_id}\nERROR: {error}"

    # Create message
    msg = MIMEMultipart()
    msg['From'] = sender_email
    msg['To'] = receiver_email
    msg['Subject'] = subject
    msg.attach(MIMEText(body, 'plain'))

    # Connect to Gmail SMTP server and send
    with smtplib.SMTP_SSL('smtp.gmail.com', 465) as server:
        server.login(sender_email, password)
        server.send_message(msg)

    print("Email sent successfully!")

In [3]:
def remove_accents(text):
    if not isinstance(text, str):
        return text
    # Normalize to NFKD (decomposes characters)
    text = unicodedata.normalize('NFKD', text)
    # Encode to ASCII bytes, ignore non-ASCII characters, then decode back to str
    return text.encode('ascii', 'ignore').decode('ascii')

In [4]:
def retrieve_boxscore(con, soup, gm_type, date, home_team, away_team):
    
    target_ids = [f"box-{home_team}-{gm_type}-basic", f"box-{away_team}-{gm_type}-basic"]
    target_tables = [soup.find("table", {"id": tid}) for tid in target_ids]
    dfs = [pd.read_html(str(tbl), header=1)[0] for tbl in target_tables]
    
    home = dfs[0]
    home['Team'] = home_team
    home['Team_type'] = 'Home'
    home['Opp'] = away_team
    home['Opp_type'] = 'Away'

    away = dfs[1]
    away['Team'] = away_team
    away['Team_type'] = 'Away'
    away['Opp'] = home_team
    away['Opp_type'] = 'Home'
    
    df = pd.concat([home, away])
    df = df[~(df.Starters.isin(['Team Totals', 'Reserves']))].fillna(0).replace("Did Not Play", "DNP")\
            .replace("Did Not Dress", "DNP").replace("Not With Team", "DNP").replace("Player Suspended", "DNP").rename(columns={"Starters": "Player", "3P": "3PM"})
    df["Player"] = df["Player"].apply(lambda x: x.encode("latin1").decode("utf-8"))
    df['Player'] = df['Player'].apply(remove_accents)
    df['Date'] = pd.to_datetime(date, format="%Y%m%d")
    df['Active'] = np.where(df['MP'] == 'DNP', 0, 1)

    df['MP'] = np.where(df.MP == 'DNP', '0:0', df.MP)
    df['mins'] = df.MP.str.split(":").str[0]
    df['secs'] = df.MP.str.split(":").str[1]
    df = df.fillna({'mins': 0, 'secs': 0})
    df['MP'] = df.mins.astype(int) + round(df.secs.astype(int) / 60, 2)
    
    for col in df.columns.difference(['Date', 'Player', 'Team', 'Team_type', 'Opp', 'Opp_type']):
        df[col] = np.where(df[col] == 'DNP', 0, df[col])
        
    for col in ['MP', 'FG%', '3P%', 'FT%', 'GmSc']:
        df[col] = df[col].astype(float)

    for col in df.columns.difference(['Date', 'Player', 'MP', 'Team', 'Team_type', 'Opp', 'Opp_type', 
                                      'FG%', '3P%', 'FT%', 'GmSc', '+/-']):
        df[col] = df[col].astype(int)

    df_pos = pd.read_csv(f"../tables/{year}/plyr_pos_xref.csv")
    df = con.execute("""SELECT * FROM df LEFT JOIN df_pos ON df.Player = df_pos.Player AND df.Team = df_pos.Team""").fetchdf()
        
    df['game_id'] = f"{date}_{away_team}_{home_team}"
    df = df[['game_id', 'Date', 'Team', 'Team_type', 'Opp', 'Opp_type', 'Player', 'Pos', 'Active', 'MP', 
             'FG', 'FGA', 'FG%', '3PM', '3PA', '3P%', 'FT', 'FTA',
             'FT%', 'ORB', 'DRB', 'TRB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'PTS',
             'GmSc', '+/-']]
    
    return df

In [5]:
def fantasy_scoring(df):
    
    # Columns we care about
    stat_cols = ['PTS', 'TRB', 'AST', 'STL', 'BLK']

    # Count how many stats are >= 10 for each player
    df['double_count'] = (df[stat_cols] >= 10).sum(axis=1)

    # Determine double-double or triple-double
    df['DD'] = (df['double_count'] >= 2).astype(int)
    df['TD'] = (df['double_count'] >= 3).astype(int)
    
    df['40Pts'] = (df['PTS'] >= 40).astype(int)
    df['50Pts'] = (df['PTS'] >= 40).astype(int)
    
    df['Fpts'] = (df.PTS * 0.5) + (df.AST * 1) + (df.TRB * 1) + (df.STL * 2) + (df.BLK * 2) + (df.TOV * -1) + (df['3PM'] * 0.5)\
                   + (df.DD * 1) + (df.TD * 2) + (df['40Pts'] * 2) + (df['50Pts'] * 2)
    
    df = df.drop(columns=['double_count', '40Pts', '50Pts'])

    return df

In [9]:
def main():
    con = duckdb.connect(database=":memory:")

    df = pd.read_csv(f"../tables/{year}/nba_schedule.csv")
    df['Date'] = pd.to_datetime(df.Date).astype(str)
    filtrd_df = df[(df.rtrvd == 0) & (df.Date < now)].reset_index(drop=True)
    display(filtrd_df)
    
    for row in range(filtrd_df.shape[0]):
        game = filtrd_df.loc[row]
        date = game.loc['Date'].replace("-", "")
        home_team = game.loc['HomeABV']
        away_team = game.loc['AwayABV']
        url = f"https://www.basketball-reference.com/boxscores/{date}0{home_team}.html"
        print(url)
        
        time.sleep(6)
        if RUN_LOCATION == 'local':
            html = requests.get(url).text
        else:
            scraper = cloudscraper.create_scraper()
            html = scraper.get(url).text
        soup = BeautifulSoup(html, "lxml")
        
        for gm_type in ['game', 'q1', 'q2', 'q3', 'q4', 'h1', 'h2']:
            if gm_type == 'game':
                file_name = 'season_gamelogs.csv'
            else:
                file_name = f'{gm_type}_season_gamelogs.csv'
                
            try:
                print(f'{gm_type.capitalize()}_Log of {date}_{away_team}_{home_team}')
                df_gm = retrieve_boxscore(con, soup, gm_type, date, home_team, away_team)
                df_gm = fantasy_scoring(df_gm)
                
                try:
                    df_cnct = pd.read_csv(f"../tables/{year}/{file_name}")
                except:
                    df_cnct = pd.DataFrame()
                    
                df_cnct = pd.concat([df_cnct, df_gm])
                df_cnct['Date'] = pd.to_datetime(df_cnct['Date'], format='mixed', errors='coerce')
                df_cnct['Date'] = df_cnct['Date'].dt.strftime("%m/%d/%Y")
                df_cnct.to_csv(f"../tables/{year}/{file_name}", index=False)
                
                if gm_type == 'game':
                    display(df_gm)

            except Exception as e:
                game_id = f"{date}_{away_team}_{home_team}"
                email(e, game_id)
                raise Exception(e)
        
        df_save_indx = df[(df.Date == game.loc['Date']) & (df.AwayABV == away_team) & (df.HomeABV == home_team)].index
        df.loc[df_save_indx, 'rtrvd'] = 1
        df.to_csv(f"../tables/{year}/nba_schedule.csv", index=False)

    con.close()
    
main()

,Date,StartTime_ET,AwayTeam,AwayABV,HomeTeam,HomeABV,Arena,AwayB2B,HomeB2B,rtrvd
0,2023-04-18,7:30p,New York Knicks,NYK,Cleveland Cavaliers,CLE,Rocket Mortgage Fieldhouse,0,0,0
1,2023-04-18,10:00p,Los Angeles Clippers,LAC,Phoenix Suns,PHO,Footprint Center,0,0,0
2,2023-04-19,7:30p,Los Angeles Lakers,LAL,Memphis Grizzlies,MEM,FedEx Forum,0,0,0
3,2023-04-19,9:00p,Miami Heat,MIA,Milwaukee Bucks,MIL,Fiserv Forum,0,0,0
4,2023-04-19,10:00p,Minnesota Timberwolves,MIN,Denver Nuggets,DEN,Ball Arena,0,0,0
...,...,...,...,...,...,...,...,...,...,...
68,2023-06-01,8:30p,Miami Heat,MIA,Denver Nuggets,DEN,Ball Arena,0,0,0
69,2023-06-04,8:00p,Miami Heat,MIA,Denver Nuggets,DEN,Ball Arena,0,0,0
70,2023-06-07,8:30p,Denver Nuggets,DEN,Miami Heat,MIA,Kaseya Center,0,0,0
71,2023-06-09,8:30p,Denver Nuggets,DEN,Miami Heat,MIA,Kaseya Center,0,0,0


https://www.basketball-reference.com/boxscores/202304180CLE.html
Game_Log of 20230418_NYK_CLE


,game_id,Date,Team,Team_type,Opp,Opp_type,Player,Pos,Active,MP,FG,FGA,FG%,3PM,3PA,3P%,FT,FTA,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,GmSc,+/-,DD,TD,Fpts
0,20230418_NYK_CLE,2023-04-18,CLE,Home,NYK,Away,Jarrett Allen,C,1,34.82,4,8,0.500,0,0,0.000,1,2,0.500,6,4,10,3,3,3,1,2,9,15.4,+16,0,0,28.5
1,20230418_NYK_CLE,2023-04-18,CLE,Home,NYK,Away,Darius Garland,PG,1,36.35,8,17,0.471,6,10,0.600,10,11,0.909,0,3,3,7,0,0,4,2,32,23.9,+29,0,0,25.0
2,20230418_NYK_CLE,2023-04-18,CLE,Home,NYK,Away,Danny Green,SG,1,19.82,1,2,0.500,1,2,0.500,0,0,0.000,0,4,4,0,2,0,1,3,3,3.0,-2,0,0,9.0
3,20230418_NYK_CLE,2023-04-18,CLE,Home,NYK,Away,Caris LeVert,SG,1,39.85,9,16,0.563,4,9,0.444,2,2,1.000,0,4,4,3,0,0,2,1,24,17.3,+23,0,0,19.0
4,20230418_NYK_CLE,2023-04-18,CLE,Home,NYK,Away,Robin Lopez,C,1,2.37,2,2,1.000,0,0,0.000,0,0,0.000,0,0,0,0,0,0,0,0,4,3.4,-3,0,0,2.0
5,20230418_NYK_CLE,2023-04-18,CLE,Home,NYK,Away,Sam Merrill,SG,0,0.00,0,0,0.000,0,0,0.000,0,0,0.000,0,0,0,0,0,0,0,0,0,0.0,0,0,0,0.0
6,20230418_NYK_CLE,2023-04-18,CLE,Home,NYK,Away,Donovan Mitchell,SG,1,38.92,6,11,0.545,2,5,0.400,3,4,0.750,1,1,2,13,2,1,2,4,17,20.5,+24,1,0,29.5
7,20230418_NYK_CLE,2023-04-18,CLE,Home,NYK,Away,Evan Mobley,PF,1,39.17,6,11,0.545,0,0,0.000,1,2,0.500,2,11,13,0,2,2,2,3,13,12.2,+15,1,0,26.5
8,20230418_NYK_CLE,2023-04-18,CLE,Home,NYK,Away,Raul Neto,PG,1,3.22,0,0,0.000,0,0,0.000,0,0,0.000,0,0,0,0,0,0,0,2,0,-0.8,-10,0,0,0.0
9,20230418_NYK_CLE,2023-04-18,CLE,Home,NYK,Away,Isaac Okoro,SF,1,2.75,0,1,0.000,0,0,0.000,0,0,0.000,1,0,1,0,0,0,2,2,0,-2.8,+2,0,0,-1.0


https://www.basketball-reference.com/boxscores/202304180PHO.html
Game_Log of 20230418_LAC_PHO


,game_id,Date,Team,Team_type,Opp,Opp_type,Player,Pos,Active,MP,FG,FGA,FG%,3PM,3PA,3P%,FT,FTA,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,GmSc,+/-,DD,TD,Fpts
0,20230418_LAC_PHO,2023-04-18,LAC,Away,PHO,Home,Nicolas Batum,PF,1,19.03,0,4,0.000,0,4,0.000,0,0,0.000,0,1,1,1,0,0,0,3,0,-3.0,-17,0,0,2.0
1,20230418_LAC_PHO,2023-04-18,LAC,Away,PHO,Home,Brandon Boston Jr.,SF,1,0.97,0,0,0.000,0,0,0.000,0,0,0.000,0,0,0,0,0,0,1,0,0,-1.0,-2,0,0,-1.0
2,20230418_LAC_PHO,2023-04-18,LAC,Away,PHO,Home,Amir Coffey,SF,1,0.97,0,0,0.000,0,0,0.000,0,0,0.000,0,0,0,0,0,0,0,0,0,0.0,-2,0,0,0.0
3,20230418_LAC_PHO,2023-04-18,LAC,Away,PHO,Home,Robert Covington,PF,0,0.00,0,0,0.000,0,0,0.000,0,0,0.000,0,0,0,0,0,0,0,0,0,0.0,0,0,0,0.0
4,20230418_LAC_PHO,2023-04-18,LAC,Away,PHO,Home,Eric Gordon,SG,1,32.33,4,12,0.333,4,9,0.444,0,0,0.000,0,2,2,2,0,0,1,4,12,4.6,-17,0,0,11.0
5,20230418_LAC_PHO,2023-04-18,LAC,Away,PHO,Home,Bones Hyland,PG,1,14.10,2,7,0.286,0,4,0.000,2,2,1.000,0,2,2,0,0,1,1,2,6,1.4,+2,0,0,6.0
6,20230418_LAC_PHO,2023-04-18,LAC,Away,PHO,Home,Kawhi Leonard,SF,1,38.83,11,20,0.550,3,5,0.600,6,7,0.857,3,5,8,7,3,0,3,1,31,29.1,-10,0,0,35.0
7,20230418_LAC_PHO,2023-04-18,LAC,Away,PHO,Home,Terance Mann,SG,1,25.62,3,3,1.000,2,2,1.000,2,2,1.000,1,1,2,0,0,0,2,4,10,6.5,-3,0,0,6.0
8,20230418_LAC_PHO,2023-04-18,LAC,Away,PHO,Home,Marcus Morris,PF,0,0.00,0,0,0.000,0,0,0.000,0,0,0.000,0,0,0,0,0,0,0,0,0,0.0,0,0,0,0.0
9,20230418_LAC_PHO,2023-04-18,LAC,Away,PHO,Home,Mason Plumlee,C,1,13.60,0,0,0.000,0,0,0.000,2,2,1.000,1,1,2,0,0,0,1,0,2,2.0,+3,0,0,2.0


...

,game_id,Date,Team,Team_type,Opp,Opp_type,Player,Pos,Active,MP,FG,FGA,FG%,3PM,3PA,3P%,FT,FTA,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,GmSc,+/-,DD,TD,Fpts
0,20230607_DEN_MIA,2023-06-07,DEN,Away,MIA,Home,Christian Braun,SG,1,19.03,7,8,0.875,0,1,0.000,1,1,1.000,2,2,4,1,1,0,1,1,15,14.5,+2,0,0,13.5
1,20230607_DEN_MIA,2023-06-07,DEN,Away,MIA,Home,Bruce Brown,SF,1,28.62,1,5,0.200,1,3,0.333,2,2,1.000,0,2,2,0,0,3,1,3,5,2.4,+8,0,0,10.0
2,20230607_DEN_MIA,2023-06-07,DEN,Away,MIA,Home,Thomas Bryant,C,1,0.48,0,0,0.000,0,0,0.000,0,0,0.000,0,0,0,0,0,0,0,0,0,0.0,0,0,0,0.0
3,20230607_DEN_MIA,2023-06-07,DEN,Away,MIA,Home,Kentavious Caldwell-Pope,SG,1,28.10,1,4,0.250,0,3,0.000,4,4,1.000,0,3,3,0,0,0,1,2,6,2.7,+12,0,0,5.0
4,20230607_DEN_MIA,2023-06-07,DEN,Away,MIA,Home,Vlatko Cancar,PF,1,0.48,0,0,0.000,0,0,0.000,0,0,0.000,0,0,0,0,0,0,0,0,0,0.0,0,0,0,0.0
5,20230607_DEN_MIA,2023-06-07,DEN,Away,MIA,Home,Aaron Gordon,PF,1,33.80,5,10,0.500,0,0,0.000,1,4,0.250,3,7,10,5,1,0,0,3,11,12.3,+16,1,0,23.5
6,20230607_DEN_MIA,2023-06-07,DEN,Away,MIA,Home,Jeff Green,PF,1,17.48,2,2,1.000,0,0,0.000,0,0,0.000,0,0,0,1,0,0,0,1,4,3.7,-1,0,0,3.0
7,20230607_DEN_MIA,2023-06-07,DEN,Away,MIA,Home,Reggie Jackson,PG,1,1.38,0,1,0.000,0,1,0.000,0,0,0.000,0,1,1,1,0,0,0,0,0,0.3,-2,0,0,2.0
8,20230607_DEN_MIA,2023-06-07,DEN,Away,MIA,Home,Nikola Jokic,C,1,43.75,12,21,0.571,1,2,0.500,7,8,0.875,3,18,21,10,0,2,3,5,32,32.6,+15,1,1,51.5
9,20230607_DEN_MIA,2023-06-07,DEN,Away,MIA,Home,DeAndre Jordan,C,0,0.00,0,0,0.000,0,0,0.000,0,0,0.000,0,0,0,0,0,0,0,0,0,0.0,0,0,0,0.0


https://www.basketball-reference.com/boxscores/202306090MIA.html
Game_Log of 20230609_DEN_MIA


,game_id,Date,Team,Team_type,Opp,Opp_type,Player,Pos,Active,MP,FG,FGA,FG%,3PM,3PA,3P%,FT,FTA,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,GmSc,+/-,DD,TD,Fpts
0,20230609_DEN_MIA,2023-06-09,DEN,Away,MIA,Home,Christian Braun,SG,1,14.75,0,1,0.000,0,0,0.000,1,2,0.500,0,1,1,0,1,1,0,1,1,1.5,-17,0,0,5.5
1,20230609_DEN_MIA,2023-06-09,DEN,Away,MIA,Home,Bruce Brown,SF,1,29.58,8,11,0.727,3,5,0.600,2,3,0.667,0,4,4,2,1,1,0,1,21,20.0,0,0,0,22.0
2,20230609_DEN_MIA,2023-06-09,DEN,Away,MIA,Home,Thomas Bryant,C,0,0.00,0,0,0.000,0,0,0.000,0,0,0.000,0,0,0,0,0,0,0,0,0,0.0,0,0,0,0.0
3,20230609_DEN_MIA,2023-06-09,DEN,Away,MIA,Home,Kentavious Caldwell-Pope,SG,1,36.02,2,5,0.400,2,5,0.400,1,2,0.500,2,2,4,2,3,2,0,2,7,10.9,+16,0,0,20.5
4,20230609_DEN_MIA,2023-06-09,DEN,Away,MIA,Home,Vlatko Cancar,PF,0,0.00,0,0,0.000,0,0,0.000,0,0,0.000,0,0,0,0,0,0,0,0,0,0.0,0,0,0,0.0
5,20230609_DEN_MIA,2023-06-09,DEN,Away,MIA,Home,Aaron Gordon,PF,1,41.70,11,15,0.733,3,4,0.750,2,3,0.667,1,6,7,6,1,0,2,4,27,24.6,+29,0,0,28.0
6,20230609_DEN_MIA,2023-06-09,DEN,Away,MIA,Home,Jeff Green,PF,1,15.55,1,1,1.000,1,1,1.000,0,0,0.000,0,0,0,0,1,0,1,1,3,2.3,-7,0,0,3.0
7,20230609_DEN_MIA,2023-06-09,DEN,Away,MIA,Home,Reggie Jackson,PG,0,0.00,0,0,0.000,0,0,0.000,0,0,0.000,0,0,0,0,0,0,0,0,0,0.0,0,0,0,0.0
8,20230609_DEN_MIA,2023-06-09,DEN,Away,MIA,Home,Nikola Jokic,C,1,37.13,8,19,0.421,3,7,0.429,4,4,1.000,2,10,12,4,3,3,3,5,23,20.2,+9,1,0,39.0
9,20230609_DEN_MIA,2023-06-09,DEN,Away,MIA,Home,DeAndre Jordan,C,0,0.00,0,0,0.000,0,0,0.000,0,0,0.000,0,0,0,0,0,0,0,0,0,0.0,0,0,0,0.0


https://www.basketball-reference.com/boxscores/202306120DEN.html
Game_Log of 20230612_MIA_DEN


,game_id,Date,Team,Team_type,Opp,Opp_type,Player,Pos,Active,MP,FG,FGA,FG%,3PM,3PA,3P%,FT,FTA,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,GmSc,+/-,DD,TD,Fpts
0,20230612_MIA_DEN,2023-06-12,DEN,Home,MIA,Away,Christian Braun,SG,1,24.28,2,4,0.500,0,0,0.000,3,6,0.500,1,2,3,1,0,1,0,0,7,6.5,0,0,0,9.5
1,20230612_MIA_DEN,2023-06-12,DEN,Home,MIA,Away,Bruce Brown,SF,1,28.15,4,14,0.286,0,5,0.000,2,4,0.500,4,2,6,1,1,0,1,1,10,4.7,0,0,0,13.0
2,20230612_MIA_DEN,2023-06-12,DEN,Home,MIA,Away,Thomas Bryant,C,0,0.00,0,0,0.000,0,0,0.000,0,0,0.000,0,0,0,0,0,0,0,0,0,0.0,0,0,0,0.0
3,20230612_MIA_DEN,2023-06-12,DEN,Home,MIA,Away,Kentavious Caldwell-Pope,SG,1,34.00,4,10,0.400,1,5,0.200,2,2,1.000,1,3,4,2,2,3,1,1,11,11.3,+7,0,0,21.0
4,20230612_MIA_DEN,2023-06-12,DEN,Home,MIA,Away,Vlatko Cancar,PF,0,0.00,0,0,0.000,0,0,0.000,0,0,0.000,0,0,0,0,0,0,0,0,0,0.0,0,0,0,0.0
5,20230612_MIA_DEN,2023-06-12,DEN,Home,MIA,Away,Aaron Gordon,PF,1,28.65,1,6,0.167,0,2,0.000,2,4,0.500,2,5,7,1,2,1,1,4,4,3.1,+8,0,0,15.0
6,20230612_MIA_DEN,2023-06-12,DEN,Home,MIA,Away,Jeff Green,PF,1,4.73,2,2,1.000,0,0,0.000,0,0,0.000,0,0,0,1,0,0,0,2,4,3.3,-5,0,0,3.0
7,20230612_MIA_DEN,2023-06-12,DEN,Home,MIA,Away,Reggie Jackson,PG,0,0.00,0,0,0.000,0,0,0.000,0,0,0.000,0,0,0,0,0,0,0,0,0,0.0,0,0,0,0.0
8,20230612_MIA_DEN,2023-06-12,DEN,Home,MIA,Away,Nikola Jokic,C,1,42.30,12,16,0.750,1,3,0.333,3,5,0.600,1,15,16,4,0,1,4,3,28,24.3,+12,1,0,33.5
9,20230612_MIA_DEN,2023-06-12,DEN,Home,MIA,Away,DeAndre Jordan,C,1,3.02,0,0,0.000,0,0,0.000,0,0,0.000,0,0,0,0,0,1,0,0,0,0.7,-2,0,0,2.0
